## Main Step
1. Look at the big picture
2. Get the data
3. Discover and visualize the data to gain insights
4. Prepare the data for Machine Learning Algorithm
5. Select a model and train it
6. Fine-tune your model
7. Present you solution
8. Launch, monitor, and maintain your system

## Important Things
- the business objective
- the current solution
- problem definition
- check the assumption

## Measure generally preferred in regression tasks
- Root Mean Squared Error (RMSE)
    - how much error the system typically makes in its prediction  
    with a higher weight for large errors
    - Euclidian norm(distance), also called L2 norm
- Mean Absolute Error (MAE)
    - If there are many outliers, you may consider using this
    - L1 norm, Manhattan norm
- In L'k' norm, the higher the norm index k, the more it focuses on large values  
and neglects small ones
- L0 norm just gives the cardinality of the vector
   

# Get the data

## Take a Quick Look at the Data Structure

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + HOUSING_PATH + "/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

## Summary of the numerical attributes
- null values are ignored

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
#plt.show()

### a few things in these histogram
- median income is not expressed in USD
- some data has been scaled and capped
- capped data can be a serious problem
    - prices never go beyond the limit(capped value)
    - solution?
        - collect the actual value
        - remove those data from training & test set
- attributes have very different scales
- tail heavy!
    - transform them to bell-shaped dist

## Create a Test Set
- no data **snooping bias**!
    - our brain is amazing pattern detection system

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

### purely random sampling ?
- this is generally fine if our dataset is **large enough**
    - but if it is not, can be lead to significant **sampling bias**
- if US population is composed of 51.3% female, 48.7% male
    - should maintain this ratio in the sample: 513 female & 487 male
- **Stratified Sampling**
- if the median imcome is important attribute to predict median housing prices
    - then the test set is representative of the various categories of incomes in the whole datasets
    - so, first we need to divide data into some stratums
    - and each stratum have a sufficient number of instances
        - or else the estimate of the stratum's importance may be biased
- we do not have too many strata, and each stratum should be large enough

In [ ]:
housing["median_income"].hist()

In [ ]:
import numpy as np

housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
housing.head()

In [ ]:
housing['income_cat'].value_counts()

In [ ]:
housing["income_cat"].hist()

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
housing['income_cat'].value_counts() / len(housing)

In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
compare_props

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop(["income_cat"], axis=1, inplace=True)

# Discover and Visualize the Data to Gain Insights
- we only exploring the training set
- if it is very large, then we may want to sample an **exploration set**

In [ ]:
housing = strat_train_set.copy()

In [ ]:
housing.plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
            s=housing["population"]/100, label="population", figsize=(10,7),
            c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
            sharex=False)
plt.legend()

### This image tells...
- housing prices are very much related to the location (close to the ocean)
- and to the population density
- can add new features that measure the proximity to the (main) cluster centers

## Looking for Correlations
- dataset is not too large
    - we can compute the **standard correlation coefficient**
    - -1 ~ 1
        - close to 1 ; strong positive correlation
        - close to -1 ; strong negative correlation
        - close to 0 ; no linear correlation

In [ ]:
corr_matrix = housing.corr()

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

### correlation coefficient
- it only measures linear corrleations
    - nothing to do with the slope
- completely miss out on nonlinear relationships
<img src="./imgs/cor_ex.png"></img>

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

### the most promising attr is the median income

In [ ]:
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

### there are some horizental straight lines caused by capping
- we may want to try removing the corresponding districts
- need to clean them up !

## Experimenting with Attributes Combination!!
- **interative process**!!
- number of rooms per household
- the total number of bedrooms **by itself is not very useful**
    - compare it to the number of rooms
- population per household

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedroom_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

# Prepare the Data for Machine Learning Algorithms

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing.head()

In [ ]:
housing_labels.head()

## Data Cleaning
- Most ML algorithm cannot work with missing features
    - Get rid of the row
    - Get rid of the col(attribute)
    - Set the values to some value

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

#### Option 1

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])

#### Option 2

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)

#### Option 3

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True)
sample_incomplete_rows

## Imputer
- handy class to take care of missing values !
- Scikit-learn Design Pattern (p.61)

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="median")

housing_num = housing.drop("ocean_proximity", axis=1)
# alternatively: housing_num = housing.select_dtypes(include=[np.number])

imputer.fit(housing_num)

imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
X = imputer.transform(housing_num) # this is ndarray

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                         index=list(housing.index.values)) # keep the index

In [ ]:
housing_tr.loc[sample_incomplete_rows.index.values]

#### put it back into DataFrame

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)
housing_tr.info()

In [ ]:
housing_tr.head()

## Handling Text and Categorical Attributes
- Most ML algorithms prefer to work with numbers anyways

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

housing_cat = housing[['ocean_proximity']]
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

#### Problem
- two nearby values are more similar than two distant values
- need **one-hot encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat)
housing_cat_1hot # sparse matrix (call toarray() to get dense ndarray)

In [ ]:
housing_cat_1hot.toarray()

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

## Custom Transformer
- fit(), transform(), fit_transform()
- TransformerMixin, BaseEstimator

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]


class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
attr_addr = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_addr.transform(housing.values)

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs, 
    columns=list(housing.columns) + ["rooms_per_household", "population_per_household"])
housing_extra_attribs.head()

#### Alternatively, we can use **FunctionTransformer** class
- let us easily create a transformer based on a transformation function
- validate=False, because the data contains non-float values

In [ ]:
from sklearn.preprocessing import FunctionTransformer

def add_extra_features(X, add_bedrooms_per_room=True):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]
    else:
        return np.c_[X, rooms_per_household, population_per_household]

attr_adder = FunctionTransformer(add_extra_features, validate=False,
                                 kw_args={"add_bedrooms_per_room": False})
housing_extra_attribs = attr_adder.fit_transform(housing.values)

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"])
housing_extra_attribs.head()

## Feature Scaling!
- ML algorithms don't perform well when the input numerical attrs have very different scales
- But, scaling the target values is generally not required
- Two common ways
    - min-max scaling
        - 0-1 range
        - (value - min)/max
        - more affected by outlier
        - **MinMaxScaler**
    - standardization
        - (value - mean)/variance
        - the result dist has unit variance
        - does not have specific range -> may be problem for some algorithms
            - ex) neural network often expect an input value 0 - 1
        - less affected by outlier
        - **StandardScaler**

## Transformation Pipelines
- Scikit-learn provides the Pipeline class 
    - to make the transformation steps executed in the right order
- **Pipeline** cunstructor takes a list of name/estimator pairs defining a sequence of steps
    - All but the last estimator must be transformers
- When we call the pipeline's **fit()** method
    - it calls **fit_transform()** sequentially on all transformers
    - passing the output of each call as the parameter to the next call
    - final estimator just calls the **fit()** method
- And, **expose the same methods as the final estimator**
    - example below shows that it can call fit_transform() because the last estimator(StandardScaler) is transformer, and it has fit_transform()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    #('attribs_adder', CombinedAttributesAdder()), # preferable option
    ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

## Join Numerical & Categorical transformation 

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared[0], housing_prepared.shape

## Select and Train a Model

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

In [ ]:
print("Labels:", list(some_labels))

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

#### median_housing_values range \$120000 ~ \$265000
- not satisfying result
- example of underfitting the training data
    - features may not provide enough info to make good predictions
    - or, the model is not powerful enough

#### Try a DecisionTreeRegressor
- powerful model
- capable of finding complex nonlinear relationships in the data

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

#### badly ovefit the data
- we don't want to touch the test set until we are ready to lunch a model 
- so, we need to use part of the training set for **training**, and part for **model validation**

## Better Evaluation Using Cross-Validation
- K-fold cross-validation
    - randomly splits the training set into 10 district subsets called **fold**
    - then trains & evaluates the model 10 times, picking up a different fold for evaluation
- scikit-learn cross-validation features expect a **utility function(greater is better)**
rather than a **cost function(lower is better)**

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

#### Random Forests
- training **many Decision Trees** on **random subsets of the features**,
    - then averaging out their predictions
- building a model on top of many other models is called **Ensemble Learning**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

#### Still Overfitting !!
- the score on the training set is still much lower than on the validation sets
- Solutions
    - simplify the model
    - constrain it (regularize it)
    - get a lot more training data

#### The Goal of these series of works is to shortlist a few promising models

## Fine-Tune Your Model
- if we got a shortlist of promising models
    - we now need to fine-tune them

#### Grid Search
- will evaluate all the possible combinations of hyperparameter values, using cross-validation
- when we have no idea what values to try
    - try out consecutive powers of 10
- refit=True (defualt, when the best estimator is found, retrain it on the whole data set)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
# pd.DataFrame(grid_search.cv_results_).head()

#### Some of the data preparation steps also can be treated as hyperparams!
- add feature or not
    - using add_bedrooms_per_room hyperparamter of our CombinedAttributesAdder transformer
- find the best way to handle outliers, missing features, feature selection, etc.

#### Randomized Search 
- if the hyperparams **search space is large**
- often prefereable to use **RandomizedSearchCV** instead
    - evaluates a given number(iteration) of **random combinations**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
rnd_search.best_params_

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

#### Ensemble Methods
- The group will often perform better than the individual Decision Trees,
- especially **if the individual models makes very different types of errors**

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # old solution
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

#### With this info
- we may want to try **dropping some of the less useful features**
    - e.g. only one ocean_proximity category is really useful, could try dropping other categories
- And we should also look at the specific errors that our system makes
    - then, try to understand why it makes them and how could fix the problem
    - **adding extra features or, getting rid of uninformatives ones, cleaning up outliers...**


#### Evaluate Your System on the Test Set!

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

#### Resist the Temptation!
- The performance usually be slightly worse than what we measured using CV if we did a lot of hyperparams tuning
    - because our system ends up fine-tuned to perform well on the validation set
    - and will not perform as well on unknown data
- Must resist the temptation to tweak the hyperparams to make the numbers look good on the test set